In [1]:
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [2]:
# Display the shapes of the data.
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of y_test: {y_test.shape}')

Shape of X_train: (50000, 32, 32, 3)
Shape of X_test: (10000, 32, 32, 3)
Shape of y_train: (50000, 1)
Shape of y_test: (10000, 1)


In [3]:
from layers_activation import ReluLayer
from layers_convolution import ConvolutionLayer
from layers_pooling import PoolLayer
from layers_flatten import Flatten
from layers_dense import DenseLayer
from loss import SoftmaxLayer
from network import Model

In [4]:
model = Model(
    ConvolutionLayer(filters=3, filter_size=3, padding=1, stride=1),
    ReluLayer(),
    PoolLayer(filter_size=2, stride=2, mode='max'),
    Flatten(),
    DenseLayer(units=100),
    SoftmaxLayer()
)

In [5]:
model.train(X_train, y_train, 0.003, batch_size=64, epochs=100)

Running epoch: 1
Z input shape in Flatten forward prop: (64, 16, 16, 3)
data output shape in Flatten forward prop: (64, 768)
W input shape in Dense forward prop: (64,)
X input shape in Dense forward prop: (768, 64)
b input shape in Dense forward prop: (100,)


ValueError: shapes (64,) and (768,64) not aligned: 64 (dim 0) != 768 (dim 0)

In [ ]:
def to_categorical(labels, num_classes):
    """
    Converts image labels to one-hot encoded binary vectors with length of num_classes.
    Arguments:
        labels -- image classification labels e.g. 1 = 'airplane', nested numpy arrays of integers.
        num_classes -- number of classes in image classification task, integer.
    Returns:
        label_vec -- binary vector representation of the class, 1 for correct label, 0's otherwise. 
    """
    # Initialize a zero matrix with the correct dimensions.
    label_vec = np.zeros((len(labels), num_classes))
    # Loop through the labels in the input.
    for i in range(len(labels)):
        # Create an new index for binary encoding, subtracting 1. 
        # An image with label [6] on a scale of 1-10, will have a 1 at index 5 in the vector.
        new_index = labels[i][0] - 1
        # Replace the 0 at the given location with a 1 for the correct class.
        label_vec[i][new_index] = 1
    return label_vec

In [ ]:
import numpy as np
y_train_mine = to_categorical(y_train, 10)

In [ ]:
y_train_mine.shape